In [1]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep
import pandas as pd
import numpy as np
import time

# Make a dataframe to store the information scrapped from Amazon.
pricelist = pd.DataFrame(columns=['Name','Price','Miner'])
 

# Function to scrap the information from Amazon.
def x(GPU):
    from selectorlib import Extractor
    import requests 
    import json 
    from time import sleep


    # Create an Extractor by reading from the YAML file
    e = Extractor.from_yaml_file('search_results.yml')

    def scrape(url):  

        headers = {
            'dnt': '1',
            'upgrade-insecure-requests': '1',
            'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-user': '?1',
            'sec-fetch-dest': 'document',
            'referer': 'https://www.amazon.com/',
            'accept-language': 'en-US;q=0.9,en;q=0.8',
        }
        # Download the page using requests
        print("Downloading %s"%url)
        r = requests.get(url, headers=headers)
        # Simple check to check if page was blocked (Usually 503)
        if r.status_code > 500:
            if "To discuss automated access to Amazon data please contact" in r.text:
                print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
            else:
                print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
            return None
        # Pass the HTML of the page and create 
        return e.extract(r.text)

    # product_data = []
    
    with open("search_results_urls.txt",'r') as urllist, open('search_results_output.jsonl','w') as outfile:        
        for url in urllist.read().splitlines():
            data = scrape(url) 
                    
            if data:
                for product in data['products']:
                    leng = len(GPU.split(" "))
                    count=0
                    for gpu in GPU.split(" "):
                        
                        if gpu.lower() in product['title'].lower():
                            count+=1
                        else:
                            pass 
                        try:
                            if len(GPU.split(" "))==count:
                                pricelist.loc[len(pricelist)] = [product['title'],float(product['price'].replace(",",'').replace("$",'')),GPU]
                        except:
                            pass    

                    product['search_url'] = url
                    print("Saving Product: %s"%product['title'])
                    json.dump(product,outfile)
                    outfile.write("\n")

                    
## Knapsack problem solver
def knapsack_unbounded_dp(items, C):
    # order by max value per item size
    items = sorted(items, key=lambda item: item[VALUE]/float(item[SIZE]), reverse=True)
 
    # Sack keeps track of max value so far as well as the count of each item in the sack
    sack = [(0, [0 for i in items]) for i in range(0, C+1)]   # value, [item counts]
 
    for i,item in enumerate(items):
        name, size, value = item
        for c in range(size, C+1):
            sackwithout = sack[c-size]  # previous max sack to try adding this item to
            trial = sackwithout[0] + value
            used = sackwithout[1][i]
            if sack[c][0] < trial:
                # old max sack with this added item is better
                sack[c] = (trial, sackwithout[1][:])
                sack[c][1][i] +=1   # use one more
 
    value, bagged = sack[C]
    numbagged = sum(bagged)
    size = sum(items[i][1]*n for i,n in enumerate(bagged))
    # convert to (iten, count) pairs) in name order
    bagged = sorted((items[i][NAME], n) for i,n in enumerate(bagged) if n)
 
    return value, size, numbagged, bagged

In [3]:
# Read GPU file which we obtained from Nicehash.
# Read the paper for the link.
miner = pd.read_csv("miner.csv")
miner['Model'] = miner['Model'].replace(['MicroBT'],'', regex=True)
miner['Model'] = miner['Model'].replace(['BITMAIN'],'', regex=True)
miner = miner.drop_duplicates(subset ="Model",keep = 'first')
miner['profit'] = miner['Net Profit'].str.split(" ").str[0]
miner['profit'] =miner['profit'].astype(float)


for gp in miner['Model'].values:
    # Make URL for each GPU
    f = open("search_results_urls.txt","w")
    f.write("https://www.amazon.com/s?k="+str(gp))
    f.close()
    # Call scrapper
    x(gp)

# Please verify the information of scrapping if there is an issue analysis will go wrong.
# Save price information.
pricelist.to_csv("prices_miner.csv")

In [7]:
# Read Miner file which we obtained from Nicehash.
miner = pd.read_csv("miner.csv")
miner['Model'] = miner['Model'].replace(['MicroBT'],'', regex=True)
miner['Model'] = miner['Model'].replace(['BITMAIN'],'', regex=True)
miner = miner.drop_duplicates(subset ="Model",keep = 'first')

# Read Prices file which we obtained from Amazon.
prices  = pd.read_csv("prices_miner.csv",index_col=0)
miner['profit'] = miner['Net Profit'].str.split(" ").str[0]
miner['profit'] =miner['profit'].astype(float)

#gpu['price'] =gpu['price'].str.replace(".","")
#miner['price'] =miner['price'].str.replace(".","")
asdprices = prices.sort_values(by = 'Price',ascending = True)
decprices = prices.sort_values(by = 'Price',ascending = False)
low = asdprices.drop_duplicates(subset ="Miner",keep = 'first')
high = decprices.drop_duplicates(subset ="Miner",keep = 'first')

miner = miner[miner['Model'].isin(list(low['Miner'].values))]
miner = miner[['Model','profit']]

miner = miner.sort_values(by = 'Model',ascending = True)
low = low.sort_values(by = 'Miner',ascending = True)
high = high.sort_values(by = 'Miner',ascending = True)


low['profit'] = miner['profit'].values
high['profit'] = miner['profit'].values
low = low[['Miner','Price','profit']]
high = high[['Miner','Price','profit']]
low['Recoverytimeindays'] = low['Price']/low['profit']
high['Recoverytimeindays'] = high['Price']/high['profit']



# Get low and high price for both products if there is a duplication.
low = low.sort_values(by = 'Recoverytimeindays',ascending = True)
high = high.sort_values(by = 'Recoverytimeindays',ascending = True)


from operator import itemgetter as iget
from itertools import product
from random import shuffle
low = low[['Miner','Price','profit']]
high = high[['Miner','Price','profit']]
low['Price'] = low['Price'].astype(int)
high['Price'] = high['Price'].astype(int)

NAME, SIZE, VALUE = range(3)
itemslow = low.values
itemshigh = high.values

capacity = 5000

# Simple Case
print("Money: ",capacity," :", knapsack_unbounded_dp(high.values,capacity)) 

Money:  5000  : (1256.6399999999942, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


In [8]:
# Variation in Money
for loop in range(1000,11000,1000):
    print("Money: ",loop,",","Recovery time: ",int(loop/knapsack_unbounded_dp(high.values, loop)[0]),knapsack_unbounded_dp(high.values, loop)) 

Money:  1000 , Recovery time:  4 (249.15000000000003, 990, 55, [(' AntMiner T15', 55)])
Money:  2000 , Recovery time:  3 (502.82999999999856, 1998, 111, [(' AntMiner T15', 111)])
Money:  3000 , Recovery time:  3 (752.459999999997, 3000, 167, [(' AntMiner S9k', 3), (' AntMiner T15', 164)])
Money:  4000 , Recovery time:  3 (1005.6599999999955, 3996, 222, [(' AntMiner T15', 222)])
Money:  5000 , Recovery time:  3 (1256.6399999999942, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])
Money:  6000 , Recovery time:  3 (1508.4899999999925, 5994, 333, [(' AntMiner T15', 333)])
Money:  7000 , Recovery time:  3 (1760.819999999991, 7000, 389, [(' AntMiner S9k', 1), (' AntMiner T15', 388)])
Money:  8000 , Recovery time:  3 (2011.3199999999895, 7992, 444, [(' AntMiner T15', 444)])
Money:  9000 , Recovery time:  3 (2264.9999999999986, 9000, 500, [(' AntMiner T15', 500)])
Money:  10000 , Recovery time:  3 (2514.1500000000096, 9990, 555, [(' AntMiner T15', 555)])


In [10]:
# Variation in time period
# We considered a constant decrease of 0.1 in profit.
# We considered high values for further analysis.

possiblesolutions = {}

# Time step
timesteps = 10
temphigh = high.copy()
for loop in range(0,timesteps):
    print("\n")
    print("Decreasing profit values for all GPUs: ",loop)
    print(knapsack_unbounded_dp(itemshigh,capacity))
    possiblesolutions[loop]= knapsack_unbounded_dp(itemshigh,capacity)
    temphigh['profit']= temphigh['profit'] - 0.1
    itemshigh = temphigh.values
print("\n")
tableoftimestep = pd.DataFrame()
for key in possiblesolutions:
    tempprofit = 0
    tempprofitlist = []
    for time in range(0,timesteps): 
        temp = 0
        for GPU in possiblesolutions[key][3]:
            temp = temp+ high[high['Miner']==GPU[0]]['profit'].values[0]*GPU[1]-0.1*time*GPU[1]
        tempprofitlist.append(temp)    
    tableoftimestep["time_step"+str(key)] = tempprofitlist
print(tableoftimestep)
sum_column = tableoftimestep.sum(axis=0)
print(sum_column)




Decreasing profit values for all GPUs:  0
(978.6399999999986, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasing profit values for all GPUs:  1
(1228.8399999999976, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasing profit values for all GPUs:  2
(1201.0400000000013, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasing profit values for all GPUs:  3
(1173.2400000000039, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasing profit values for all GPUs:  4
(1145.440000000002, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasing profit values for all GPUs:  5
(1117.639999999998, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasing profit values for all GPUs:  6
(1089.8399999999951, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasing profit values for all GPUs:  7
(1062.0400000000043, 5000, 278, [(' AntMiner S9k', 2), (' AntMiner T15', 276)])


Decreasin